In [69]:
import pandas as pd
import numpy as np
import math
import csv

spambase = pd.read_csv('spambase.csv',header=None)
columns_names = ["word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", "word_freq_our", 
                    "word_freq_over", "word_freq_remove", "word_freq_internet", "word_freq_order", "word_freq_mail",
                    "word_freq_receive", "word_freq_will", "word_freq_people", "word_freq_report", "word_freq_addresses",
                    "word_freq_free", "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit", 
                    "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", "word_freq_hp", 
                    "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs", 
                    "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", 
                    "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct", 
                    "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project", "word_freq_re",
                    "word_freq_edu", "word_freq_table", "word_freq_conference", "char_freq_;", "char_freq_(", 
                    "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_#", "capital_run_length_average", 
                    "capital_run_length_longest", "capital_run_length_total", "is_spam"]
spambase.rename(columns={0:"word_freq_make", 1:"word_freq_address", 2:"word_freq_all", 3:"word_freq_3d", 4:"word_freq_our", 
                    5:"word_freq_over", 6:"word_freq_remove", 7:"word_freq_internet", 8:"word_freq_order", 9:"word_freq_mail",
                    10:"word_freq_receive", 11:"word_freq_will", 12:"word_freq_people", 13:"word_freq_report", 14:"word_freq_addresses",
                    15:"word_freq_free", 16:"word_freq_business", 17:"word_freq_email", 18:"word_freq_you", 19:"word_freq_credit", 
                    20:"word_freq_your", 21:"word_freq_font", 22:"word_freq_000", 23:"word_freq_money", 24:"word_freq_hp", 
                    25:"word_freq_hpl", 26:"word_freq_george", 27:"word_freq_650", 28:"word_freq_lab", 29:"word_freq_labs", 
                    30:"word_freq_telnet", 31:"word_freq_857", 32:"word_freq_data", 33:"word_freq_415", 34:"word_freq_85", 
                    35:"word_freq_technology", 36:"word_freq_1999", 37:"word_freq_parts", 38:"word_freq_pm", 39:"word_freq_direct", 
                    40:"word_freq_cs", 41:"word_freq_meeting", 42:"word_freq_original", 43:"word_freq_project", 44:"word_freq_re",
                    45:"word_freq_edu", 46:"word_freq_table", 47:"word_freq_conference", 48:"char_freq_;", 49:"char_freq_(", 
                    50:"char_freq_[", 51:"char_freq_!", 52:"char_freq_$", 53:"char_freq_#", 54:"capital_run_length_average", 
                    55:"capital_run_length_longest", 56:"capital_run_length_total", 57:"is_spam"},inplace=True)

class Lista():
    def __init__(self, k):
        self.lista = []
        self.num_neighbours = k

    def add_pivo(self, pivo):
        self.pivo = pivo
    
    def adiciona(self, new):
        #print(new, self.pivo)
        distance = new.calculate_distance(self.pivo)

        if len(self.lista) != 0 and distance > self.lista[-1].calculate_distance(self.pivo):
            return 
        if len(self.lista) == self.num_neighbours:
            self.lista.pop(-1)
        self.lista.append(new)
        self.lista.sort(key=lambda x: x.calculate_distance(self.pivo))

    def qtd_exemplares_por_spam(self):
        epe = {}
        for i in self.lista:
            if epe.get(i.spam) == None:
                epe[i.spam] = 1
            else:
                epe[i.spam] += 1
        return epe
        

    def define_spam(self):
        qtd_por_spam = self.qtd_exemplares_por_spam()
        #print("qtd_por_especie:", qtd_por_especie)
        maior = list(qtd_por_spam.keys())[0]
        self.pivo.set_spam(maior)
        return True

        

class Node ():
    def __init__(self, dataArr):
        self.dataArr = dataArr

    def set_spam(self, spam):
        self.spam = spam
    
    def calculate_distance(self, other_node):
        #try creating and traversing an array you dumb fuck
        #print(self.dataArr, other_node.dataArr)
        dA = np.subtract(self.dataArr, other_node.dataArr)
        
        dA = dA * dA        
        return math.sqrt(np.sum(dA))
    
    def show(self):
        print("Mail Characteristics:", self.dataArr,
        "is Spam:", self.spam)
        

class KNN ():
    def __init__(self, k, tamanho_treino):
        self.k = k
        self.tamanho_treino = tamanho_treino
        self.distance_matrix = []
        self.nodes = []
        self.training_population = None
        self.test_population = None

    def main (self):
        self.set_populations(self.tamanho_treino)
        self.training_algorithm()
        self.test_algorithm()
    
    def set_populations(self, training_population_size):
        spam = pd.read_csv("spambase.csv", names = columns_names)
        spam = spam.sample(frac=1).reset_index(drop=True)
        self.training_population = spam.iloc[:training_population_size]
        self.test_population = spam.iloc[training_population_size:]
        
    def training_algorithm(self):
        self.nodes = self.make_nodes(self.training_population)
        
    def make_nodes(self, df):
        new_nodes = []
        mailData = []
        for index,row in df.iterrows():
            array = self.dir_to_array(df, index)
            new_node = Node(array)
            new_node.set_spam(df["is_spam"][index])
            new_nodes.append(new_node)
        return new_nodes        

    def dir_to_array(self, df, index):
        a = []
        for i in df:
            if i != "is_spam":
                a.append(df[i][index])
        return a

    def test_algorithm(self):
        a = b = c = d = 0
        test_nodes = self.make_nodes(self.test_population)
        i = 0
        for node in test_nodes:
            minha_lista = Lista(self.k)

            minha_lista.add_pivo(node)
            for node in self.nodes:
                minha_lista.adiciona(node)
            minha_lista.define_spam()
            
            if minha_lista:
                self.nodes.append(minha_lista.pivo)
            if self.test_population.get(["is_spam"]).iloc[i,0] == 0:
                if minha_lista.pivo.spam == 0:
                    a += 1
                else: 
                    b += 1
            else:
                if minha_lista.pivo.spam == 0:
                    d += 1
                else: 
                    c += 1
            i += 1
        print("Matriz de confusão 🥴")
        print("\t\t Negative \t Positive")
        print("Negative \t", a, "\t\t", b)
        print("Positive \t", c, "\t\t", d)
        

if __name__ == "__main__":
    a = 3220
    knn = KNN(7, 4500)
    knn.main()

Matriz de confusão 🥴
		 Negative 	 Positive
Negative 	 59 		 6
Positive 	 27 		 9
